In [1]:
from pyproj import Proj
import numpy as np

In [2]:
# Southampton is in UTM zone 30
utm_zone = 30
projection = Proj(proj='utm', zone=utm_zone, ellps='WGS84')

In [3]:
np.divmod(14.51, 1.)

(14.0, 0.5099999999999998)

In [4]:
def deg_mins_to_degrees(degrees, minutes, direction):
    d = degrees + (minutes / 60.)
    if direction in 'SW':
        d = -d
    return d

def degrees_to_deg_mins(degrees):
    sign = 1
    if degrees < 0:
        sign = -1
        degrees = -degrees
    whole, frac = np.divmod(degrees, 1.)
    return (sign, int(whole), frac * 60.)

def display_deg_mins(N, E):
    sign, n_deg, n_mins = degrees_to_deg_mins(N)
    ns = 'N' if (sign > 0) else 'S'
    sign, e_deg, e_mins = degrees_to_deg_mins(E)
    ew = 'E' if (sign > 0) else 'W'
    print("{}° {:.5}' {}, {}° {:.5}' {}".format(
        n_deg, n_mins, ns, e_deg, e_mins, ew) )

In [5]:
def generate_obstacle_course(start_ll, end_ll):
    start_utm = np.array(projection(start_ll[1], start_ll[0]))
    end_utm = np.array(projection(end_ll[1], end_ll[0]))
    
    midpoint = (start_utm + end_utm)/2
    start_end = end_utm - start_utm
    #start_end_distance = np.sqrt((start_end**2).sum())
    start_end_distance = np.linalg.norm(start_end)
    metre_along = start_end / start_end_distance
    metre_across = np.array([-metre_along[1], metre_along[0]])
    
    corners = [ll[::-1] for ll in [projection(p[0], p[1], inverse=True) for p in [
        midpoint - (10 * metre_across) - (75 * metre_along),
        midpoint + (10 * metre_across) - (75 * metre_along),
        midpoint + (10 * metre_across) + (75 * metre_along),
        midpoint - (10 * metre_across) + (75 * metre_along),
    ]]]
    centre_zone = [ll[::-1] for ll in [projection(p[0], p[1], inverse=True) for p in [
        midpoint - (10 * metre_across) - (25 * metre_along),
        midpoint + (10 * metre_across) - (25 * metre_along),
        midpoint + (10 * metre_across) + (25 * metre_along),
        midpoint - (10 * metre_across) + (25 * metre_along),
    ]]]
    return corners, centre_zone

In [6]:
marker154 = deg_mins_to_degrees(50, 49.347, 'N'), deg_mins_to_degrees(1, 18.771, 'W')
marker155 = deg_mins_to_degrees(50, 49.237, 'N'), deg_mins_to_degrees(1, 18.834, 'W')
marker156 = deg_mins_to_degrees(50, 49.245, 'N'), deg_mins_to_degrees(1, 18.673, 'W')

In [7]:
course1, zone1 = generate_obstacle_course(marker155, marker156)
course2, zone2 = generate_obstacle_course(marker154, marker155)

In [8]:
import folium
calshot_latlon = [50.821483, -1.311233]

course1_poly = folium.features.PolygonMarker(course1, fill_opacity=0.)
zone1_poly = folium.features.PolygonMarker(zone1, fill_opacity=0.25, popup='1')

course2_poly = folium.features.PolygonMarker(course2, fill_opacity=0.)
zone2_poly = folium.features.PolygonMarker(zone2, fill_opacity=0.25, popup='2')

In [9]:
map_osm = folium.Map(location=calshot_latlon, zoom_start=17)

folium.Marker(marker154, popup='154').add_to(map_osm)
folium.Marker(marker155, popup='155').add_to(map_osm)
folium.Marker(marker156, popup='156').add_to(map_osm)

map_osm.add_child(course1_poly)
map_osm.add_child(zone1_poly)

map_osm.add_child(course2_poly)
map_osm.add_child(zone2_poly)

In [10]:
import csv
with open('obstacle_course_1.csv', 'w') as f:
    cw = csv.writer(f)
    cw.writerow(['point', 'latitude', 'longitude'])
    for p, coord in zip('ABCD', course1):
        cw.writerow([p, *coord])
    
    for p, coord in zip('EFGH', zone1):
        cw.writerow([p, *coord])
    
    cw.writerow(['marker155', *marker155])
    cw.writerow(['marker156', *marker156])

In [11]:
import csv
with open('obstacle_course_2.csv', 'w') as f:
    cw = csv.writer(f)
    cw.writerow(['point', 'latitude', 'longitude'])
    for p, coord in zip('ABCD', course2):
        cw.writerow([p, *coord])
    
    for p, coord in zip('EFGH', zone2):
        cw.writerow([p, *coord])
    
    cw.writerow(['marker154', *marker154])
    cw.writerow(['marker155', *marker155])

Convert obstacle zones to degrees and minutes for the safety boats.

In [12]:
for corner in zone1:
    display_deg_mins(*corner)

50° 49.235' N, 1° 18.774' W
50° 49.245' N, 1° 18.775' W
50° 49.247' N, 1° 18.733' W
50° 49.237' N, 1° 18.732' W


In [13]:
for corner in zone2:
    display_deg_mins(*corner)

50° 49.307' N, 1° 18.803' W
50° 49.303' N, 1° 18.787' W
50° 49.277' N, 1° 18.802' W
50° 49.281' N, 1° 18.818' W
